In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import talib

# Đọc dữ liệu
file_path = r'D:\startup\fx\merged_data_files.csv'
data = pd.read_csv(file_path)
# Đổi tên cột
data.columns = ['ngay', 'gio', 'Open', 'High', 'Low', 'Close', 'Volume']
# Kết hợp hai cột ngày và giờ
data['timestamp'] = pd.to_datetime(data['ngay'] + ' ' + data['gio'])
# Xóa hai cột ngay và gio ban đầu
data.drop(columns=['ngay', 'gio'], inplace=True)
# Đặt cột timestamp làm index
data.set_index('timestamp', inplace=True)
# Sắp xếp dữ liệu theo thời gian
data.sort_index(inplace=True)
data.reset_index(drop=False, inplace=True)
print(data.head())


            timestamp     Open     High      Low    Close  Volume
0 2024-01-01 17:00:00  1.10427  1.10429  1.10425  1.10429       0
1 2024-01-01 17:01:00  1.10429  1.10429  1.10429  1.10429       0
2 2024-01-01 17:02:00  1.10429  1.10429  1.10429  1.10429       0
3 2024-01-01 17:03:00  1.10429  1.10429  1.10426  1.10426       0
4 2024-01-01 17:04:00  1.10426  1.10431  1.10424  1.10425       0


In [8]:
# Tính toán các chỉ báo kỹ thuật nếu chưa có
if 'SMA' not in data.columns:
    data['SMA'] = talib.SMA(data['Close'], timeperiod=20)
if 'EMA' not in data.columns:
    data['EMA'] = talib.EMA(data['Close'], timeperiod=20) 
if 'RSI' not in data.columns:
    data['RSI'] = talib.RSI(data['Close'], timeperiod=14)
if 'Bollinger_High' not in data.columns:
    data['Bollinger_High'], data['Bollinger_Mid'], data['Bollinger_Low'] = talib.BBANDS(data['Close'], timeperiod=20)
if 'MACD' not in data.columns:
    data['MACD'], data['MACD_Signal'], data['MACD_Hist'] = talib.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

    
data["Next_Close"] = data["Close"].shift(-1)
# # Xóa các hàng có giá trị NaN trong nhãn
data = data.dropna()
# # Lưu dữ liệu đã xử lý
data.to_csv('modified_data.csv', index=False)
# Khám phá dữ liệu
print(data.head())



             timestamp     Open     High      Low    Close  Volume       SMA  \
33 2024-01-01 17:37:00  1.10431  1.10432  1.10431  1.10431       0  1.104298   
34 2024-01-01 17:38:00  1.10431  1.10432  1.10431  1.10432       0  1.104299   
35 2024-01-01 17:39:00  1.10431  1.10435  1.10431  1.10431       0  1.104299   
36 2024-01-01 17:40:00  1.10431  1.10431  1.10431  1.10431       0  1.104299   
37 2024-01-01 17:41:00  1.10431  1.10437  1.10431  1.10432       0  1.104304   

         EMA        RSI  Bollinger_High  Bollinger_Mid  Bollinger_Low  \
33  1.104300  53.148588        1.104298       1.104298       1.104298   
34  1.104302  55.731822        1.104299       1.104299       1.104299   
35  1.104303  52.608060        1.104299       1.104299       1.104299   
36  1.104303  52.608060        1.104299       1.104299       1.104299   
37  1.104305  55.500718        1.104304       1.104304       1.104304   

        MACD  MACD_Signal     MACD_Hist  Next_Close  
33  0.000010     0.000009 

In [6]:
# Chia dữ liệu thành tập huấn lulabelsện và tập kiểm tra
features = ['SMA', 'EMA', 'RSI', 'Bollinger_High', 'Bollinger_Low', 'MACD']
X = data[features]
y = data['Next_Close']

# Loại bỏ hàng cuối cùng nếu vẫn còn NaN sau khi điền
X = X[:-1]
y = y[:-1]

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Xây dựng và huấn luyện mô hình với ít cây hơn để tăng tốc độ
model = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(max_depth=10, n_estimators=50, random_state=42)

In [7]:
#Lưu trữ model
import pickle
file_name = "RandomForestRegressorModel"
with open(file_name, 'wb') as file:
    pickle.dump(model, file)

In [4]:
# # Hiển thị kết quả dự đoán và giá trị thực tế
# plt.figure(figsize=(14, 7))
# plt.plot(y_test.index, y_test, label='Actual Prices')
# plt.plot(y_test.index, y_pred, label='Predicted Prices')
# plt.legend()
# plt.xlabel('Timestamp')
# plt.ylabel('Price')
# plt.title('Actual vs Predicted Prices')
# plt.show()

# # Biểu đồ giá thực tế
# plt.figure(figsize=(14, 7))
# plt.plot(y_test.index, y_test, label='Actual Prices', color='blue', linewidth=2)
# plt.xlabel('Date')
# plt.ylabel('Price')
# plt.title('Actual Prices')
# plt.legend()
# plt.grid(True)
# plt.show()

# # Biểu đồ giá dự đoán
# plt.figure(figsize=(14, 7))
# plt.plot(y_test.index, y_pred, label='Predicted Prices', color='red', linestyle='--', linewidth=2)
# plt.xlabel('Date')
# plt.ylabel('Price')
# plt.title('Predicted Prices')
# plt.legend()
# plt.grid(True)
# plt.show()
